In [2]:
# TODOs: Operate DB Class
# TODOs: Use SQLite
# TODOs: Implement BM25

from utils.elastic_db import ElasticDB

# INIT DB OBJECT
PORT = "http://localhost:9200"
INDEX_NAME = "news_cc"

news_db = ElasticDB(elastic_port=PORT, elastic_index=INDEX_NAME)
wiki_db = ElasticDB(elastic_port=PORT, elastic_index="knowledge")

ModuleNotFoundError: No module named 'utils'

In [564]:
### LOAD DATASETS ###
import json

data = [json.loads(ln) for ln in open("../../data/train_cmv_cleaned.jsonl")]
topics = [json.loads(ln) for ln in open("../../data/claim_topics.jsonl")]

ex_retrieval = [json.loads(ln) for ln in open("../../data/wiki_doc_retrieved_from_op_train.jsonlist")]
ex_ranked = [json.loads(ln) for ln in open("../../data/selected_evidence.jsonl")]

conan = [json.loads(ln) for ln in open("../../data/CONAN.json")]

In [565]:
len(data)

3456

In [566]:
### SAMPLE OUTPUT ###
import random

# TODOs: Replicate Output, Passage Evidence Retrieval
# TODOs: Replicate Output, Passage Ranking

_ = random.randint(0, 1000)
ranked = ex_ranked[_]
retireval = ex_retrieval[_]

retireval

{'tid': 't3_1qafze',
 'retrieval_results': [{'retrieved_document_titles': ['South African criminal law',
    'Congenital disorder',
    'Insanity defense',
    'Treason in Arthurian legend',
    'Psychiatric and mental health nursing'],
   'query': 'egregious crimes mental defects commit',
   'sentence': 'People with mental defects who commit egregious crimes should not be punished any differently . '},
  {'retrieved_document_titles': ['Kehar Singh',
    'Sentence (law)',
    'Adam Yahiye Gadahn',
    '68th Academy Awards',
    'Lionel Jospin'],
   'query': 'egregious crime a crime horrifying',
   'sentence': "I do n't know if there 's a legal term for what I have in mind for ` egregious crime ' - more of an intuitive notion of a crime that is simply horrifying , that shocks you about what people are capable of . "},
  {'retrieved_document_titles': ['South African criminal law',
    'Ages of consent in Europe',
    'Ghulam Azam',
    'LGBT rights in the United States',
    'Insanity de

In [567]:
# TODOs: News Data
# TODOs: Ranking
# TODOs: Ranking, Cosine
# TODOs: Research Evidence Retireval: Context Aware, Neural Retrieval
# TODOs: Stance
# TODOs: Target ADUs: Premises, Claims, discard non-ADUs, thus reducing noise over retreival 
# TODOs: Paralellise

# TERMS => Input Argument : args : arg
# TERMS => Argument Discourse Units : adu : adu_prem : adu_claim
# TERMS => Evidence : ev
# TERMS => Counter Evidence : counta_ev

In [568]:
### SUBJECT ARG ###
import random
import re

# Note: 340; Gender Equality

_ = random.randint(0, 1000)
print(_)

claim = data[_]["titles"]
arg = data[_]["arguments"]
claim, arg

622


('Welfare you receive isnt your money and things like food stamps that make welfare nondiscretionary are a good idea.',
 'A social safety net is a good idea and money is spent with the intent of providing a specific kind of help. In many places this includes a cash handout. I believe this is wrong on many levels. The money may be misspent. This idea is NOT based on the assumption that poor people are all stupid or drug addicts. Its simply the fact that theres no way of ensuring that the cash is used for its intended purpose. The best assumption you can make is that people would be so desperately craving the thing you give them extra money for that they would almost certainly spend it on that but people may end up endure forms of squalor that were far from what the well intentioned policy makers had in mind. From the governments perspective money is also being spent inefficiently. Lets say a program wanted to ensure that people were well fed. Theyd have to find a cash amount that would 

In [569]:
### NLP FUNCTIONS ###

def sentences_segment(doc):
    return [i for i in re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', doc)]

def tokeniser(doc):
    return re.findall(r"\w+(?:'\w+)?|[^\w\s]", doc)

# Test Statements
tokeniser("hello, my name is Josh!")
sentences_segment("hello, my name is Josh! How are you doing today? I'm curious ... will this line seperate? I'm not so sure Dr. Evil")

['hello, my name is Josh! How are you doing today?',
 "I'm curious ... will this line seperate?",
 "I'm not so sure Dr. Evil"]

In [570]:
from utils.keyphrase_extraction import extract_keyphrase

# TODOs: Fix Sentencizer
# TODOs: Tokeniser Function
# TODOs: Domain Restrict - polarising social and political debate (Class labelling); Note: currently, open-domain. Score Highly Polarised Discussions.
# TODOs: Bag of Topics, Concepts for Topic Labelling
# TODOs: News, Political, Sociology and 'Good', 'Positive' counter-evidence Knowledge Base.
# TODOs: Consider parsing and normalising knowledge; extracting core ADU arguments, premises, evidence and claims.
# TODOs: Sentence Segment Function
# TODOs: Semantic Retrieval ** 
# TODOs: Fix Sentence parsing
# TODOs: Parameterise Index-DB in use
# TODOs: Implement as a Class

### RETRIEVER ###

# TODOs: Filter ADUs; Strong and disputable arguments (Premise rakning, NLI, Argument Similarity (counter stance))
db = news_db

ad_units = [i for i in re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', arg)]

queries = []
retrieved_ev = []

for adu in ad_units:
    toks = re.findall(r"\w+(?:'\w+)?|[^\w\s]", adu)

    if len(toks) <= 8:
        continue

    print(adu)    
    kp = extract_keyphrase(adu, n_kp=3)
    query = ", ".join(i for i in kp)
    
    # TODOs: Re-init DB with smaller passage size
    titles = [i["_source"]["document"]["title"] for i in db.search(query_=query, k=10)]
    evidence = [i["_source"]["document"]["text"] for i in db.search(query_=query, k=10)]

    retrieved_ev.append({
        "argument_discourse_unit": adu, 
        "retrieved_documents_titles": titles,
        "adu_keyphrases": query,
        "retrieved_evidence": evidence,
        "merged_evidence": ", ".join(ln for ln in evidence)
    })

A social safety net is a good idea and money is spent with the intent of providing a specific kind of help.


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.075s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.018s]


In many places this includes a cash handout.


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.115s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.008s]


I believe this is wrong on many levels.


...

INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.064s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.008s]


Being well fed healthy comfortably sheltered continually educated to eventually earn the life you want are more important than not being insulted.


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.017s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.004s]


If anyone can enlighten me on what its like to be offended by a government policy assuming youre stupid or malevolent especially if you dont pay taxes that would be helpful.


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.086s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.010s]


In [571]:
### PASSAGE RANKING; KEYWORD OVERLAP ###

### SAMPLE SINGLE OBJECT ###
adu_idx = 0
ev_idx = 0

# TODOs: For each ADU, Rank Merged Evidence using Keyword Overlap and Filter for Contrasting Stance

# Extract ADU and retrieved Evidence set, given by the ADU index
evidence = sentences_segment(retrieved_ev[adu_idx]["merged_evidence"])
adu = retrieved_ev[adu_idx]['argument_discourse_unit']
adu_kp = retrieved_ev[adu_idx]['adu_keyphrases']

# Sample single evidence item, from merged, given by the evidence_index
adu, adu_kp, evidence[ev_idx]

('A social safety net is a good idea and money is spent with the intent of providing a specific kind of help.',
 'social safety net, good idea, money',
 'Even some proponents of expanding the social safety net oppose the idea, fearing it would siphon money from existing programs.')

In [239]:
# TODOs: Speed-up, Parrelleise, Yield
def overlap_score(evidence_kp, adu_kp):
    score = 0
    
    # Split Keyphrase into components, scoring partial units as overlap
    for i in evidence_kp:
        for j in i.split():
            # Ensure string value, to enact .find
            if " ".join(adu_kp).find(j) != -1: score += 1
            
            else: continue
    
    return score

ev_units = evidence
adu_kp = extract_keyphrase(adu)

adu_ev_overlap = []

kp_1 = ['sex', 'relationship', 'opportunity'] 
kp_2 = ['better sex']

overlap_score(kp_2, kp_1)

for ev_unit in evidence:
    #print(ev_unit)
    toks = tokeniser(ev_unit)

    # Exprimental Value
    if len(toks) <= 8:
        continue
    
    ev_unit_kp = extract_keyphrase(ev_unit)
    kp_overlap = overlap_score(evidence_kp=ev_unit_kp, adu_kp=adu_kp)
    
    adu_ev_overlap.append({
        "adu": adu, 
        "adu_kp": adu_kp,
        "ev_unit": ev_unit,
        "ev_unit_kp": ev_unit_kp, 
        "kp_overlap": kp_overlap
        
        })
        
adu_ev_overlap


[{'adu': 'Hello! Let me preface by saying I dont believe there is a better sex.',
  'adu_kp': ['better sex'],
  'ev_unit': "The sex isn't automatically better just because you're in a relationship, but it has more of an opportunity to get really good.",
  'ev_unit_kp': ['sex', 'relationship', 'opportunity'],
  'kp_overlap': 1},
 {'adu': 'Hello! Let me preface by saying I dont believe there is a better sex.',
  'adu_kp': ['better sex'],
  'ev_unit': "The mutual trust and understanding you share with your partner will lead to better sex, but that's not the only reason sex can be better when you're in a relationship.",
  'ev_unit_kp': ['better sex', 'mutual trust', 'relationship'],
  'kp_overlap': 2},
 {'adu': 'Hello! Let me preface by saying I dont believe there is a better sex.',
  'adu_kp': ['better sex'],
  'ev_unit': "Here are three explanations for why sex can feel so much more satisfying when you're in a relationship. 1 Communication is scientifically proven to lead to better sex."

In [562]:
### OVERLAP RANKED EVIDENCE ###

adu_ev_overlap.sort(key=lambda y: y["kp_overlap"], reverse=True)
adu_ev_overlap

### FILTER IRRELEVANT EVIDENCE ###
overlapping = [i for i in adu_ev_overlap if i["kp_overlap"] !=0]

len(adu_ev_overlap), len(overlapping)
overlapping


[{'adu': 'Hello! Let me preface by saying I dont believe there is a better sex.',
  'adu_kp': ['better sex'],
  'ev_unit': 'These simple ideas and techniques could help both you and your lover enjoy better sex. 1 / 10 Getty Images/Caiaimage Think beyond the thrust., A mother-of-one believes that her job as a sex worker makes her a better parent.',
  'ev_unit_kp': ['better sex', 'sex worker', 'better parent'],
  'kp_overlap': 4},
 {'adu': 'Hello! Let me preface by saying I dont believe there is a better sex.',
  'adu_kp': ['better sex'],
  'ev_unit': "Here are three explanations for why sex can feel so much more satisfying when you're in a relationship. 1 Communication is scientifically proven to lead to better sex.",
  'ev_unit_kp': ['better sex', 'sex', 'relationship'],
  'kp_overlap': 3},
 {'adu': 'Hello! Let me preface by saying I dont believe there is a better sex.',
  'adu_kp': ['better sex'],
  'ev_unit': 'Afterplay also makes way for better sex and increases the frequency of sex

In [614]:
### ASSERT SAME STANCE ###
from detection.stance_classifier import sentence_stance, compare_stance

# TODOs: Ensure KPs Extracts are constrained to 1 unit
opposing_stance = []
for i in overlapping:
    adu = i["adu"]
    target = " ".join(i for i in i["adu_kp"])
    ev_unit = i["ev_unit"]

    print(ev_unit)
    ev_stance = compare_stance(ev_unit, ev_aspect, target)
    adu_stance = sentence_stance(adu, target)
    
    if ev_stance != adu_stance:
        opposing_stance.append((ev_unit, ev_stance, adu_stance))

    else: continue


ModuleNotFoundError: No module named 'detection'

In [613]:
# import spacy
# from spacy.matcher import PhraseMatcher
# from fuzzywuzzy import fuzz, process

# # TODOs: Package as a Module
# # TODOs: Handle Negation (Polarity shifters)
# # TODOs: Review Unsuperived Approach; Consider adveanced patterns and common-sence knowledge

# nlp = spacy.load("en_core_web_sm")

# sentence = "I hate abortion rights. Abortions should be banned."
# sentence_2 = "I like abortion rights. I belive we should keep them."
# sentence_3 = "I hate tennis. People should play tennis more often"

# ### STANCE SCORING ###

# # TODOs: https://www.cs.uic.edu/~liub/FBS/opinion-mining-final-WSDM.pdf 
# # TODOs: Pattern based Negation
# # TODOs: Semantic Orientation of an opinion (Claim)
# # TODOs:Group synonyms of 'features', 'targets'

# phrase_matcher = PhraseMatcher(nlp.vocab)

# ### SENTIMENT LEXICONS ###
# pos = [w.replace("\n", "") for w in open("../../data/lexicon/positive_lex.txt")]
# neg = [w.replace("\n", "") for w in open("../../data/lexicon/negative_lex.txt")]
# polarity_shifters = [w.replace("\n", "") for w in open("../../data/lexicon/shifter_lexicon.txt")]

# ### STANCE: ASPECT-SEMANTIC ORIENTATION ###
# def extract_aspect(sentence, n_gram):
#     aspects = extract_keyphrase(str(sentence))[0]

#     return nlp(aspects)

# def index_aspect(phrase, aspect, sentence):    
#     patterns = [nlp(aspect)]
#     phrase_matcher.add(phrase, None, *patterns)

#     start = 0
#     stop = 0

#     matched_phrases = phrase_matcher(sentence)
#     for i in matched_phrases:
#         _, start, stop = i
        
#     return start, stop

# # TODOs: Implement Polarity Shifters, Simple
# # TODOs: Implement Polarity Shifters, Complex, Verb Patterns
# def stance_score(start, stop, sentence):
#     pos_score = 0.0
#     neg_score = 0.0

#     score = 0
#     for idx, tok in enumerate(sentence):
#         if idx == start or idx == stop:
#             continue

#         # TODOs: Implement Polarity Shift
#         # TODOs: Experiement with descriptive term + keyphrase aspects
#         # TODOs: ABSA https://www.kaggle.com/code/phiitm/aspect-based-sentiment-analysis
#         # Use external libaray: Textblob
        
#         k = 8
#         # Negation Rules
#         shifted_tok = None
#         shifted_toks = []

#         if (tok.dep_ == "neg") or (tok.dep_ in polarity_shifters):
#             #Shift to Negative
#             if idx <= k:
#                 if idx < start: neg_score += 1/(start - idx)
#                 else: neg_score += 1/(idx - stop)**0.5

#             if shifted_tok != None and shifted_tok in neg:
#                 print(shifted_tok.text)
#                 # Shift to Positive
#                 if idx < start: pos_score += 1/(start - idx)
#                 elif idx > start: pos_score += 1/(idx - stop)**0.5
#                 else: continue

#         # Aspect Sentement Orientation
#         if tok.text in pos:
#             if tok in shifted_toks:
#                 continue
            
#             if idx < start: pos_score += 1/(start - idx)
#             else: pos_score += 1/(idx - stop)**0.5

#         if tok.text in neg:
#             if tok in shifted_toks:
#                 continue

#             if idx <= start: neg_score += 1/(start - idx)
#             else: neg_score += 1/(idx - stop)**0.5
    
#     score = pos_score - neg_score /(pos_score + neg_score + 1)

#     return score

# def overlap_score(evidence_kp, adu_kp):
#     score = 0
    
#     # Split Keyphrase into components, scoring partial units as overlap
#     for i in evidence_kp:
#         for j in i.split():
#             # Ensure string value, to enact .find
#             if " ".join(adu_kp).find(j) != -1: 
#                 score += 1
#                 token = j
            
#             else: continue
    
#     return score

# def get_overlapping_token(evidence_kp, adu_kp):
#     for i in evidence_kp:
#         overlap_tokens = []
#         for j in i.split():
#             if " ".join(adu_kp).find(j) != -1: 
#                 overlap_tokens.append(j) 
            
#         return " ".join(i for i in overlap_tokens)

# def sentence_stance(sentence, aspect):
#     sentence = nlp(sentence)

#     start, stop = index_aspect("aspects", aspect, sentence)
#     score = stance_score(start, stop, sentence)

#     # Add Neutral
#     #stance = {"claim": sentence, "stance": "PRO", "aspect": aspect} if score > 0 else {"claim": sentence, "stance": "CON", "aspect": aspect}
    
#     return "PRO" if score > 0 else "CON"

# def fuzzy_match(target, evidence_unit):

#     overlapping_aspect = process.extractOne(target, ev.split())[0]
#     score = overlapping_aspect[1]

#     overlapping_aspect = nlp(re.sub(r'[^\w]', ' ', overlapping_aspect))

#     return overlapping_aspect, score

# def compare_stance(ev_unit, evidence_aspect, adu_target):
#     # Note: Already identified mathcing or partially matching Aspects. 

#     # Get the overlapping evidence aspect-target.
#     overlapping_target, score = fuzzy_match(target=adu_aspect, evidence_unit=ev)
    
#     # Get position of the overlapping_target
#     start, stop = index_aspect("OVERLAP", nlp(overlapping_target), nlp(ev_unit))

#     # Assert Stance towards evidence aspect
#     score = stance_score(start, stop, nlp(ev_unit))
    
#     return "PRO" if score > 0 else "CON"

# ev = "These simple ideas and techniques could help both you and your lover enjoy sex. 1 / 10 Getty Images/Caiaimage Think beyond the thrust."
# ev_aspect = "sex", "relationship", "opportunity"

# adu = 'Hello! Let me preface by saying I dont believe there is a better sex.'
# adu_aspect = "better sex"

# print(sentence_stance("The mutual trust and understanding you share with your partner will lead to better sex, but that's not the only reason sex can be better when you're not in a relationship.", adu_aspect))
# print(compare_stance(ev, ev_aspect, adu_aspect))


PRO
PRO


In [560]:
opposing_stance

[('None, Jake: This Better Not Bite Me in the Ass, name of your sex tape.',
  'CON',
  'PRO'),
 ('In fact, a study published by National Commission on Aging found women, in particular, said sex in their 70s was at least as satisfying or more satisfying physically than it was in their 40s., The Australian sex worker: “I was disgusted that he would equate sex with his wife with ‘taking one for the team’. “I was further disgusted by the way he gaslit his wife when she was firmly in the right.',
  'CON',
  'PRO'),
 ('Jake: This Better Not Bite Me in the Ass, name of your sex tape.',
  'CON',
  'PRO')]

In [561]:
k = 1
top_k = opposing_stance[0:k]

top_k

[('None, Jake: This Better Not Bite Me in the Ass, name of your sex tape.',
  'CON',
  'PRO')]

In [597]:
test_1 = "Let me preface by saying I dont believe there is not a better sex.'"

description = []
for tok in nlp(test_1):

    if tok.text == "sex":
        print(tok.text)
        aspect = tok
        
        for child in aspect.children:
            description.append([child.text for child in i if (not child.is_stop and not child.is_punct and (child.pos_ == "ADJ" or child.pos_ == "VERB" or child.dep_ == "neg"))])

        # for anc in aspect.ancestors:
        #     description.append([anc.text for anc in i if (not anc.is_stop and not anc.is_punct and (anc.pos_ == "ADJ" or anc.pos_ == "VERB" or anc.dep_ == "neg"))])

description

sex


[['Let', 'preface', 'saying', 'nt', 'believe', 'better'],
 ['Let', 'preface', 'saying', 'nt', 'believe', 'better']]

In [505]:
# from spacy.matcher import DependencyMatcher, Matcher
# matcher = Matcher(vocab=nlp.vocab)
# matcher

# # Matching Rule: Pronouns with Verbs that follow them
# aspect = "better sex"
# patterns = [
#     [{"DEP": "neg"}, {"LOWER": aspect}],
#     [{"DEP": "neg"}, {"POS": "ADJ"}, {"LOWER": aspect}],
#     [{"POS": "VERB"}, {"POS": "ADJ"}, {"LOWER": aspect}],
#     [{"LOWER": aspect.lower()}]
# ]

# test = nlp("Hello! Let me preface by saying I dont believe there is a not better sex.")
# test_2 = nlp("These simple ideas and techniques could help both you and your lover enjoy better sex.")

# matcher.add("test", patterns=patterns)
# result = matcher(test_2, as_spans=True)

# result

# # for tok in test:
# #     print(tok.i, tok, tok.pos_, tok.dep_, tok.head.i, sep="\t")

[]

In [485]:
# negation_tokens = [tok for tok in test if tok.dep_ == 'neg']
# negation_head_tokens = [token.head for token in negation_tokens]

# for token in negation_head_tokens:
#     print(token.text, token.dep_, token.head.text, token.head.pos_, [child for child in token.children])

believe ccomp saying VERB [I, do, nt, is]
better amod sex NOUN [not]


In [466]:
# from spacy.matcher import DependencyMatcher

# doc = "These simple ideas and techniques could help both you and your lover enjoy better sex. "

# dep_matcher = DependencyMatcher(vocab=nlp.vocab)
# dep_pattern = [{'RIGHT_ID': 'verb', 'RIGHT_ATTRS': {'POS': 'VERB'}},
#                  {'LEFT_ID': 'verb', 'REL_OP': '>', 'RIGHT_ID': 'subject', 'RIGHT_ATTRS': {'DEP': 'nsubj'}},
#                  {'LEFT_ID': 'verb', 'REL_OP': '>', 'RIGHT_ID': 'd_object', 'RIGHT_ATTRS': {'DEP': 'dobj'}}
#                 ]

# # Add the pattern to the matcher under the name 'nsubj_verb'
# dep_matcher.add('nsubj_verb', patterns=[dep_pattern])

# # Apply the DependencyMatcher to the Doc object under 'doc'; Store the result 
# # under the variable 'dep_matches'.
# dep_matcher.add('nsubj_verb_dobj', patterns=[dep_pattern])

# # Call the variable to examine the output
# dep_matches = dep_matcher(nlp(doc))

# for match in dep_matches:
    
#     # Take the first item in the tuple at [0] and assign it under
#     # the variable 'pattern_name'. This item is a spaCy Lexeme object.
#     pattern_name = match[0]
    
#     # Take the second item in the tuple at [1] and assign it under
#     # the variable 'matches'. This is a list of indices referring to the
#     # Doc object under 'doc' that we just matched.
#     matches = match[1]

#     # Because we now have two patterns for matching which return lists of
#     # different length, e.g. lists with two indices for 'nsubj_verb' and
#     # lists with three indices for 'nsubj_verb_dobj', we must now define
#     # conditional criteria for handling these lists.
#     if len(matches) > 2:
        
#         # Let's unpack the matches list into variables for clarity
#         verb, subject, dobject = matches[0], matches[1], matches[2]
    
#         # Print the matches by first fetching the name of the pattern from the 
#         # Vocabulary object. Next, use the 'subject' and 'verb' variables to 
#         # index the Doc object. This gives us the actual Tokens matched. Use a
#         # tabulator ('\t') and some stops ('...') to separate the output.
#         print(nlp.vocab[pattern_name].text, '\t', doc[subject], '...', doc[verb], '...', doc[dobject])
        
#     # Alternative condition with just two items in the list.
#     else:
        
#         # Let's unpack the matches list into variables for clarity
#         verb, subject = matches
    
#         # Print the matches by first fetching the name of the pattern from the 
#         # Vocabulary object. Next, use the 'subject' and 'verb' variables to 
#         # index the Doc object. This gives us the actual Tokens matched. Use a
#         # tabulator ('\t') and some stops ('...') to separate the output.
#         print(nlp.vocab[pattern_name].text, '\t', doc[subject], '...', doc[verb])



nsubj_verb 	 m ...   ... d
nsubj_verb_dobj 	 m ...   ... d


In [7]:
#     pos_score = 0.0
#     neg_score = 0.0

#     # Pattern Match
#     phrase_matcher = PhraseMatcher(nlp.vocab)

    # compound_words = []    
    # for i in aspects:
    #     compound_word = ""
    #     if i.pos_ in ["NOUN", "PROPN"]:
    #         comps = "".join([str(j) for j in i.children if j.dep_ == "compound"])
    #         if comps:
    #             compound_word = comps + " " + str(i)
    #             compound_words.append(compound_word)

    # aspects_ = []
    # aspects_.extend(compound_words)
    # aspects_.extend(aspects)

#     for idx, tok in enumerate(sentence):

#         if idx == start or idx == stop - 1:
#             continue

#         # Polarity Shift
#         # NEAR parameter, k
#         k = 5
#         if tok.dep_ == "neg":
#             if tok.text in pos:
#                 # Shift to Negative
#                 if idx <= k:
#                     if idx < start: neg_score += 1/(start - idx)
#                     else: neg_score += 1/(idx - stop)**0.5
        
#             if str(tok.head.text) in neg:
#                 # Shift to Positive
#                 if idx < start: pos_score += 1/(start - idx)
#                 else: pos_score += 1/(idx - stop)**0.5

#         if str(tok.text) in pos:
#             if idx < start: pos_score += 1/(start - idx)
#             else: pos_score += 1/(idx - stop)**0.5

#         if str(tok.text) in neg:
#             if idx < start: neg_score += 1/(start - idx)
#             else: neg_score += 1/(idx - stop)**0.5

#     result = pos_score - neg_score /(pos_score - neg_score + 1)
#     stance = ""

#     neg_score, pos_score
#     stance = {"claim": sentence, "stance": "PRO", "aspect": aspect} if result > 0 else {"claim": sentence, "stance": "CON", "aspect": aspect}

#     return stance

# ### TEST STATEMENT ###

# id = random.randint(0, 1000)
# claim =  nlp("I do not believe abortion should be legal")

# print(sentence_stance(claim))



In [ ]:
# print(kp)

# def count(string, substring):
#     n = len(substring)
#     cnt = 0
#     for i in range(len(string) - n):
#         if string[i:i+n] == substring:
#             cnt += 1
#     return cnt

# def hasIntersection(a, b):
#         score = 0
#         a = set(a) 
#         b = set(b)
#         if a.intersection(b):
#             score += 1

#         return score

# def overlap_score(count_kp):
#     score = 0

#     for i in count_kp:
#         score += count(sent_kps, count_kp)

#     return score

# scores = []
# for sent in merged_sentences:
#     if sent:
#         # TODOs: Try-Catch, Handel Errors
#         kp = [i for i in extract_keyphrase(sent, n_kp=3)]
#         scores.append((kp, overlap_score(kp)))

# print(sent_kps)
# scores

In [59]:
# print(sent_kp)

view, arguments, logic, line


In [ ]:
### ATTACKING PEMISES ###

# from BERT_adu_classifier import predict

# premises = []
# for sent in sentences:
#     prediction = predict(sent)
    
#     if prediction == "premise":
#         premises.append(sent)

In [26]:
# # Argument Level

# results = []

# _ = 90
# hate_ = sample_hate[0]["conan"][_]["hateSpeech"]

# # queries = []
# # sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', hate_)

# results = []

# # TODOs: Refactor key_BERT as Class for multiple oututs
# #kp_ = (exctract_keyphrase(hate_, n_gram=1, n_kp=1), exctract_keyphrase(hate_, n_gram=2, n_kp=1), exctract_keyphrase(hate_, n_gram=3, n_kp=1))

# kp_ = (exctract_keyphrase(hate_, n_gram=5, n_kp=1))

# query = " ,".join(i for i in kp_)
# query

# res = [i["_source"]["document"]["title"] for i in news_db.search(query_=query, k=5)]

# results.append({
#     "argument_sentence": hate_, 
#     "query": query, 
#     "retireved_documents": res,

# })

# results


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.367s]


[{'argument_sentence': 'Fuck Islam and Fuck ISIS. We should repatriate all Muslims. They are all a bunch of violent criminals who do not share our values and never will. Without them Britain will be a better, safer place.',
  'query': 'fuck isis repatriate muslims',
  'retireved_documents': ['‘Assalamu Alaykom:’ peace be upon you',
   'How We Got to Here: A Charlottesville Reading List',
   'De-radicalization can work for former ISIS fighters',
   'How to Get Assyrians to Stay',
   'How to Get Christians to Stay']}]

In [27]:
# results = []
# res = [i["_source"]["document"]["title"] for i in wiki_db.search(query_=query, k=5)]

# results.append({
#     "argument_sentence": hate_, 
#     "query": query, 
#     "retireved_documents": res,

# })

# results

INFO:elastic_transport.transport:POST http://localhost:9200/knowledge/_search [status:200 duration:0.249s]


[{'argument_sentence': 'Fuck Islam and Fuck ISIS. We should repatriate all Muslims. They are all a bunch of violent criminals who do not share our values and never will. Without them Britain will be a better, safer place.',
  'query': 'fuck isis repatriate muslims',
  'retireved_documents': ['Refuting ISIS',
   'Refuting ISIS',
   'Take Off (2017 film)',
   'Terrorism in India',
   'Tareq Kamleh']}]

In [ ]:
# #queries
# query = "the executive the presidential system legislative branches partisan unproductive"
# res = search_text(wiki_ev, query_=query, k=3)
# res

[{'_index': 'knowledge',
  '_id': 'kbZa5YIBRqC1428DaQUk',
  '_score': 33.672318,
  '_ignored': ['document.text.keyword'],
  '_source': {'document': {'id': '3408791',
    'source': 'wikipedia',
    'title': 'Ruling party',
    'text': "In parliamentary systems, the majority in the legislature also controls the executive branch of government, thus leaving no possibility of opposing parties concurrently occupying the executive and legislative branches of government. In other systems, such as in an American style presidential system, the party of the president does not necessarily also have a legislative majority. A 'ruling party' is also used to describe the party of one-party states, such as the Chinese Communist Party in the People's Republic of China. In his political manifesto 'The Green Book', the late Libyan leader Muammar al-Gaddafi attacked the ability of the ruling party, using it as a basis for his opposition to partisan politics. None"}}},
 {'_index': 'knowledge',
  '_id': 'JQ3

In [ ]:
# import re

# print(", ".join(re.sub(r'\[\[(?:[^|\]]*\|)?([^\]]*)]]', "", i["_source"]["document"]["title"]).strip("[]") for i in res))

Ruling party, Fusion of powers, Student governments in the United States


In [ ]:
# ### TEST SEARCH ###

# def search_text(es, query_, k=5):
#     results = es.search(
#         index = es.elastic_index,
#         query = {
#             "match": {
#                 "document.text": query_,
#                 },
#         },
#         size=k)

#     hits = results["hits"]["hits"]
#     doc_ids = [row['_source']["document"]["id"] for row in hits]

#     return hits

# def search_topic(es, topic, k=5):
#     results = es.search(
#         index = es.elastic_index,
#         body= {
#             "size": k,
#             "query": {
#                 "match": {
#                     "document.title": topic,
#         }}})

#     hits = results["hits"]["hits"]
#     doc_ids = [row['_source']["document"]["id"] for row in hits]

#     title = hits[0]["_source"]["document"]["title"]
#     text = hits[0]["_source"]["document"]["text"]

#     return {
#         "title": title,
#         "text": text
#     }

# query = "government emails"
# text = search_text(wiki_ev, query_=query, k=10)

# print(text)


In [ ]:
### SEARCH FUNCTION ###

# TODOs: Implement as Class-DB object

# def search_text(db, query_, k=5):
#     results = db.search(
#         index = es.elastic_index,
#         query = {
#             "match": {
#                 "document.text": query_,
#                 },
#         },
#         size=k)

#     hits = results["hits"]["hits"]
#     doc_ids = [row['_source']["document"]["id"] for row in hits]

#     return hits